In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils

In [2]:
def trainingModel(dataLoader):
    for epoch in range(10):
        ave_loss=0.0
        print("Epoch ",epoch+1)
        for i,data in enumerate(dataLoader,0):
            inputs,target =data
            #inputs = inputs.float()
            target = target.float()
            optimizer.zero_grad()
            output=model(inputs)
            #output = output.long()
            target = target.unsqueeze(1)
            loss=criteria(output,target)
            loss.backward()
            optimizer.step()
            ave_loss+=loss.item()
        print("Loss for Epoch ",epoch+1," is ",(ave_loss/x_train.shape[0]))

In [3]:
#change the dataframe to numpy array to tensor array
def changeToTensor(dataFrame):
    dataFrame= (dataFrame-dataFrame.min())/(dataFrame.max()-dataFrame.min())
    x = dataFrame.values
    x= x.astype('float64')
    x= torch.Tensor(x)
    return x

In [4]:
train_data = pd.read_csv('train.txt',sep=',')
test_data = pd.read_csv('test.txt',sep=',')

In [5]:
train_data.head()

,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004793,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004772,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004757,1


In [6]:
### create a model with pytorch#####
class Net(nn.Module):
    def __init__(self,size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(size,1000)
        self.fc6 = nn.Linear(1000,1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = (self.fc6(x))
        return x
    def predict(self,x):
        pred = (self.forward(x))
        return pred

In [7]:
train_numpy = train_data.values
labels = train_numpy[:,-1]
labels= labels.astype('float64')
labels= torch.Tensor(labels)
train_sub = train_data.drop(['date','Occupancy'],axis=1)
x_train = changeToTensor(train_sub)

In [8]:
dataset= utils.TensorDataset(x_train,labels)
dataLoader = utils.DataLoader(dataset)

In [9]:
x_train.shape, labels.shape

(torch.Size([8143, 5]), torch.Size([8143]))

In [10]:
model = Net(x_train.shape[1])

In [11]:
model

Net(
  (fc1): Linear(in_features=5, out_features=1000, bias=True)
  (fc6): Linear(in_features=1000, out_features=1, bias=True)
)

In [12]:
criteria= nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

In [13]:
trainingModel(dataLoader)

Epoch  1
Loss for Epoch  1  is  0.3056931179870186
Epoch  2
Loss for Epoch  2  is  0.1717629633366484
Epoch  3
Loss for Epoch  3  is  0.17167802993842635
Epoch  4
Loss for Epoch  4  is  0.13384857882215936
Epoch  5
Loss for Epoch  5  is  0.12732173308247832
Epoch  6
Loss for Epoch  6  is  0.13121899693223912
Epoch  7
Loss for Epoch  7  is  0.11636467858938246
Epoch  8
Loss for Epoch  8  is  0.09822978176657204
Epoch  9
Loss for Epoch  9  is  0.07015691389997472
Epoch  10
Loss for Epoch  10  is  0.08048189180982351


In [14]:
test_numpy = test_data.values
output1 = test_numpy[:,-1]
output1=output1.astype('float64')
output1 = torch.Tensor(output1)
test_sub = test_data.drop(['date','Occupancy'],axis=1)
x_test = changeToTensor(test_sub)

In [17]:
predictions = model(x_test)

In [18]:
predictions

tensor([[17.8336],
        [17.7773],
        [17.8348],
        ...,
        [ 8.0755],
        [ 7.4072],
        [ 4.7949]], grad_fn=<AddmmBackward>)